In [1]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import MT5TokenizerFast
import json
import pandas as pd
from data_utils.utils import *

/home/nlp/anaconda3/envs/vietnq-ner/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MAX_LENGTH = 1024
model_name = 'google/mt5-base'
tokenizer = MT5TokenizerFast.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/nlp/anaconda3/envs/vietnq-ner/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Dataset: ViSquad v1.1

In [3]:
visquad_ds_json = {}

with open('datasets/ViSquadv1.1/train_ViQuAD.json') as f:
    visquad_ds_json['train'] = json.load(f)
with open('datasets/ViSquadv1.1/dev_ViQuAD.json') as f:
    visquad_ds_json['dev'] = json.load(f)
with open('datasets/ViSquadv1.1/test_ViQuAD.json') as f:
    visquad_ds_json['test'] = json.load(f)

len(visquad_ds_json['train']['data']), len(visquad_ds_json['dev']['data']), len(visquad_ds_json['test']['data'])

(138, 18, 18)

In [4]:
ds_dict = {
    'instruction': [],
    'input': [],
    'answer': [], 
    'split': []
}

for split_name, split_ds in visquad_ds_json.items():
    for data_topic in split_ds['data']:
        for data_para in data_topic['paragraphs']:
            text = data_para['context']
            for qa in data_para['qas']:
                question = qa['question']
                answer = qa['answers'][0]['text']
                ds_dict['input'].append(text)
                ds_dict['instruction'].append(question)
                ds_dict['answer'].append(answer)
                ds_dict['split'].append(split_name)

In [5]:
df = pd.DataFrame(ds_dict)
df

,instruction,input,answer,split
0,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Lâm Bá Kiệt,train
1,Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà ...,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Thủ tướng,train
2,"Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ...",Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa,train
3,Sự kiện quan trọng nào đã diễn ra vào ngày 20/...,"Năm 1954, ông được giao nhiệm vụ Trưởng phái đ...","bản Hiệp định đình chỉ chiến sự ở Việt Nam, Ca...",train
4,Chức vụ mà Phạm Văn Đồng đảm nhiệm tại Hội ngh...,"Năm 1954, ông được giao nhiệm vụ Trưởng phái đ...",Trưởng phái đoàn Chính phủ,train
...,...,...,...,...
23069,Nếu những yêu cầu mà bộ nhớ Squid không thể th...,Các yêu cầu trang được gửi cho tầng máy Squid ...,được gửi qua các máy chủ cân bằng tải (load-ba...,test
23070,Cần làm gì để tăng tốc độ trả lời cho người ch...,Các yêu cầu trang được gửi cho tầng máy Squid ...,các trang được kết xuất cho người chưa đăng nh...,test
23071,Các máy chủ cân bằng tải gửi yêu cầu cho máy c...,Các yêu cầu trang được gửi cho tầng máy Squid ...,kết xuất trang dùng dữ liệu từ CSDL,test
23072,Để giải quyết các yêu cầu thì máy chủ web có n...,Các yêu cầu trang được gửi cho tầng máy Squid ...,gửi lại những trang được yêu cầu và kết xuất t...,test


In [6]:
# create new column in df that measure number of tokens in input and instruction and answer
df['input_len'] = df['input'].apply(lambda x: len(tokenizer(x)['input_ids']))
df['instruction_len'] = df['instruction'].apply(lambda x: len(tokenizer(x)['input_ids']))
df['answer_len'] = df['answer'].apply(lambda x: len(tokenizer(x)['input_ids']))

In [7]:
df.describe()

,input_len,instruction_len,answer_len
count,23074.000000,23074.000000,23074.000000
mean,387.504681,31.839430,22.476554
std,152.812110,10.366111,23.381766
min,181.000000,4.000000,2.000000
25%,285.000000,24.000000,6.000000
50%,347.000000,31.000000,14.000000
75%,444.000000,38.000000,30.750000
max,3318.000000,108.000000,360.000000


In [8]:
# only get those has context length less than 512 tokens
df = df[df['input_len'] < 512]
df

,instruction,input,answer,split,input_len,instruction_len,answer_len
0,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Lâm Bá Kiệt,train,295,53,6
1,Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà ...,Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Thủ tướng,train,295,43,6
2,"Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ...",Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4...,Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa,train,295,33,21
3,Sự kiện quan trọng nào đã diễn ra vào ngày 20/...,"Năm 1954, ông được giao nhiệm vụ Trưởng phái đ...","bản Hiệp định đình chỉ chiến sự ở Việt Nam, Ca...",train,235,24,70
4,Chức vụ mà Phạm Văn Đồng đảm nhiệm tại Hội ngh...,"Năm 1954, ông được giao nhiệm vụ Trưởng phái đ...",Trưởng phái đoàn Chính phủ,train,235,35,11
...,...,...,...,...,...,...,...
23069,Nếu những yêu cầu mà bộ nhớ Squid không thể th...,Các yêu cầu trang được gửi cho tầng máy Squid ...,được gửi qua các máy chủ cân bằng tải (load-ba...,test,333,44,38
23070,Cần làm gì để tăng tốc độ trả lời cho người ch...,Các yêu cầu trang được gửi cho tầng máy Squid ...,các trang được kết xuất cho người chưa đăng nh...,test,333,34,40
23071,Các máy chủ cân bằng tải gửi yêu cầu cho máy c...,Các yêu cầu trang được gửi cho tầng máy Squid ...,kết xuất trang dùng dữ liệu từ CSDL,test,333,39,18
23072,Để giải quyết các yêu cầu thì máy chủ web có n...,Các yêu cầu trang được gửi cho tầng máy Squid ...,gửi lại những trang được yêu cầu và kết xuất t...,test,333,31,39


In [9]:
df.loc[:, 'prompt'] = df.apply(prompt_input, axis=1)

/tmp/ipykernel_2706835/1894269207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'prompt'] = df.apply(prompt_input, axis=1)


In [10]:
df['prompt_len'] = df['prompt'].apply(lambda x: len(tokenizer(x)['input_ids']))
df.describe()

/tmp/ipykernel_2706835/442272497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prompt_len'] = df['prompt'].apply(lambda x: len(tokenizer(x)['input_ids']))


,input_len,instruction_len,answer_len,prompt_len
count,19608.000000,19608.000000,19608.000000,19608.000000
mean,339.357711,31.648664,21.623164,423.006375
std,75.354180,10.330319,22.233965,76.576828
min,181.000000,5.000000,2.000000,246.000000
25%,277.000000,24.000000,6.000000,360.000000
50%,327.000000,30.000000,13.000000,411.000000
75%,397.000000,38.000000,30.000000,481.000000
max,511.000000,108.000000,360.000000,633.000000


In [11]:
dataset_instruct = Dataset.from_pandas(df[['prompt', 'answer']], preserve_index=False)
dataset_instruct

Dataset({
    features: ['prompt', 'answer'],
    num_rows: 19608
})

In [12]:
def tokenize(samples, tokenizer, input_field, label_field, max_length=MAX_LENGTH):
    # Tokenize
    tokenized_input = tokenizer(samples[input_field], max_length=max_length, padding=False, truncation=True, add_special_tokens=True)
    tokenized_label = tokenizer(samples[label_field], max_length=max_length, padding=False, truncation=True, add_special_tokens=True)

    return {
        'input_ids': tokenized_input['input_ids'],
        'attention_mask': tokenized_input['attention_mask'],
        'labels': tokenized_label['input_ids']
    }

print("> Tokenizing dataset...")
dataset_tokenized = dataset_instruct.map(
    lambda x: tokenize(x, tokenizer, "prompt", "answer"),
    batched=True,
    remove_columns=["prompt", "answer"],
)

print(">> Dataset is ready:")
print(dataset_tokenized)

> Tokenizing dataset...


Map: 100%|██████████| 19608/19608 [00:03<00:00, 5446.67 examples/s]

>> Dataset is ready:
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 19608
})


In [14]:
print(
    tokenizer.decode(dataset_tokenized[0]['input_ids'], skip_special_tokens=False)
)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. ### Instruction: Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm? ### Input: Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm). ### Response:</s>


In [15]:
print(
    tokenizer.decode(dataset_tokenized[0]['labels'], skip_special_tokens=False)
)

Lâm Bá Kiệt</s>


In [16]:
output_repo = 'vi-mT5-QA-ViQuAD_v1.1'
print(f"> Pushing to hub: {output_repo}")
dataset_tokenized.push_to_hub(output_repo)

> Pushing to hub: vi-mT5-QA-ViQuAD_v1.1


Uploading the dataset shards: 100%|██████████| 1/1 [00:06<00:00,  6.63s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/nqv2291/vi-mT5-QA-ViQuAD_v1.1/commit/1eb66561822bd0225853c79706adba917dfefe13', commit_message='Upload dataset', commit_description='', oid='1eb66561822bd0225853c79706adba917dfefe13', pr_url=None, pr_revision=None, pr_num=None)

## Pretrain data